### Настройки + загрузка данных

In [ ]:
import pandas as pd
import ast
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from tqdm import tqdm
import folium
from IPython.display import IFrame

In [ ]:
# Load the country table
country_table_path = 'STRANY.txt'
country_df = pd.read_csv(country_table_path, delimiter='\t', encoding='utf-8')

# Load the main file
df = pd.read_csv('Merged_fem_groups_with_user_chars_city_names.csv', low_memory=False)

# Check the structure of the loaded data
print(country_df.head())
print(df.head())

### Подготовка и геокодинг

city_names это уникальные названия городов из моего файла

In [ ]:
# Extract the 'city.name' column
city_names = df['city.name']


# Total number of entries
total_entries = len(city_names)

# Drop entries where city name is missing
city_names = city_names.dropna()

# Number of entries with specified city
entries_with_city = len(city_names)

# Calculate the share of people who specified their city
share_with_city = (entries_with_city / total_entries) * 100
print(f"Share of people who specified their city: {share_with_city:.2f}%")

В файле ```STRANY.txt``` есть список названий стран (полных и сокрвщенных) и их переводы на английский, создадим словарь для того, чтобы с его помощью перевести названия стран из документа, который у нас получился из ВК.

In [ ]:
# Создание словаря для перевода
# Используем как `name`, так и `fullname` в качестве ключей
translation_dict = {}
for _, row in country_df.iterrows():
    if pd.notna(row['name']):
        translation_dict[row['name'].strip()] = row['english'].strip()
    if pd.notna(row['fullname']):
        translation_dict[row['fullname'].strip()] = row['english'].strip()

# Функция перевода страны с использованием словаря
def translate_country_name(russian_name):
    if not russian_name or pd.isna(russian_name):
        return ''  # Возвращаем пустую строку для пропусков
    return translation_dict.get(russian_name.strip(), russian_name)  # Если нет перевода, возвращаем оригинал

# Извлечение названий стран
def extract_country_name(country_str):
    try:
        country_dict = ast.literal_eval(country_str)
        return country_dict.get('title', None)  # Извлекаем поле 'title', если оно есть
    except (ValueError, SyntaxError, KeyError, AttributeError) as e:
        print(f"Error parsing country string: {e}")
        return None

Название страны в ```Merged_fem_groups_with_user_chars_city_names.csv``` содержится в колонке `country` в формате словаря, из этого словаря нам нужно только поле `title` с, собственно, названием страны (на русском). Далее мы переводим это название с помощью нашего файла с сайта Лебедева: https://www.artlebedev.ru/country-list/tab/

In [ ]:
# Извлекаем название страны в отдельную колонку
df['country_name'] = df['country'].apply(extract_country_name)

# Пропуски оставляем пустыми строками
df['country_name'] = df['country_name'].fillna('')

# Применяем перевод
df['country_translated'] = df['country_name'].apply(translate_country_name)

# Проверяем результаты
print(df[['country', 'country_name', 'country_translated']].head())

Для более простой работы от оригинального датафрейма со всеми пользователями оставляем только табличку с их городами и странами (где есть). Дубликаты городов для визуализации нам не нужны, избавляемся от них.

In [ ]:
city_names_and_countries = df[['city.name', 'country_translated']]

unique_city_names = city_names_and_countries.drop_duplicates(subset='city.name')
print(f"Number of unique cities: {len(unique_city_names)}")

Для визуализации нам нужно *геокодировать* наши города, то есть нужны их широта и долгота. Это сделано с помощью пакета `geopy`, он будет кодировать города с использованием их стран (где есть). Надеюсь, это поможет избежать поиска пермского города Оса во Франции.

In [ ]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from tqdm import tqdm

# Initialize geolocator with a user agent
geolocator = Nominatim(user_agent="city_mapper")

# Rate limiter to respect the geocoding service's rate limits
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

# Prepare dictionaries to store results
geocoded_data = {}

# Geocode each unique city name with country information
for index, row in tqdm(unique_city_names.iterrows(), desc="Geocoding cities", total=unique_city_names.shape[0]):
    city = row['city.name']
    country = row['country_translated']
    try:
        # Include country name in the geocoding request
        location = geocode(f"{city}, {country}")
        if location:
            # Extract country from the address
            address = location.raw.get('display_name', '')
            country_extracted = address.split(',')[-1].strip()
            geocoded_data[city] = {
                'latitude': location.latitude,
                'longitude': location.longitude,
                'country': country_extracted
            }
        else:
            geocoded_data[city] = {
                'latitude': None,
                'longitude': None,
                'country': None
            }
    except Exception as e:
        geocoded_data[city] = {
            'latitude': None,
            'longitude': None,
            'country': None
        }

# Convert geocoded_data to DataFrame
geocoded_df = pd.DataFrame.from_dict(geocoded_data, orient='index').reset_index()
geocoded_df.rename(columns={'index': 'city.name'}, inplace=True)

print(f"Total unique geocoded cities: {len(geocoded_df)}")
print(geocoded_df)

city_names_df это датафрейм с уникальными городами из моего датасета

In [ ]:
# Convert geocoded_data to DataFrame
geocoded_df = pd.DataFrame.from_dict(geocoded_data, orient='index')
geocoded_df.reset_index(inplace=True)
geocoded_df.rename(columns={'index': 'city.name'}, inplace=True)

# Merge the geocoded data back to the original data
city_names_df = city_names.to_frame().reset_index(drop=True)
merged_df = city_names_df.merge(geocoded_df, on='city.name', how='left')

Почистим/причешем немного данные & посчитаем, у скольких людей указан русский город (среди всех людей с указанным городом)

In [ ]:
# Total entries with valid geocoding results
valid_geocoded_entries = merged_df.dropna(subset=['latitude', 'longitude', 'country'])

# Number of entries with Russian cities
entries_with_russian_city = valid_geocoded_entries[valid_geocoded_entries['country'] == 'Россия'].shape[0]

# Calculate the share
share_with_russian_city = (entries_with_russian_city / entries_with_city) * 100
print(f"Share of people who specified a Russian city: {share_with_russian_city:.2f}% out of the ones with the city")

valid_geocoded_entries  это датафрейм, в котором оставлены только уникальные города, их страны и координаты, полученные *геокодированием*. Загрузим Васин датасет:

In [ ]:
# Load the new dataset
df_new = pd.read_stata('socialmedia_data_main.dta')

# Extract the 'city_name_eng' column
new_city_names = df_new['city_name_eng']
new_coords = df_new[['lon', 'lat']]

# Handle missing values
new_city_names = new_city_names.dropna()

# Total number of entries in the new dataset
total_new_entries = len(new_city_names)

# Display the first few city names
print("First few city names from the new dataset:")
print(new_city_names.head())

Создадим новый датасет с городами и их координатами по Васиным данным:

In [ ]:
new_geocoded_df = pd.concat([new_coords, new_city_names], axis=1)
new_geocoded_df['country'] = 'Russian Federation' # все васины гороа из России
new_geocoded_df = new_geocoded_df.drop_duplicates(subset='city_name_eng') #убираем дубликаты городов
# Rename columns
new_geocoded_df.rename(columns={
    'city_name_eng': 'city.name',
    'lat': 'latitude',
    'lon': 'longitude'
}, inplace=True) #Переназываем столбцы для единообразия с предыдущими

Соберем мои и васины данные в один датасет

In [ ]:
# Convert new_geocoded_data to DataFrame
#new_geocoded_df = pd.DataFrame.from_dict(new_geocoded_data, orient='index')
#new_geocoded_df.reset_index(inplace=True)
#new_geocoded_df.rename(columns={'index': 'city.name'}, inplace=True)



# Combine with the previously geocoded data
combined_geocoded_df = pd.concat([geocoded_df, new_geocoded_df], ignore_index=True)

# Remove duplicates in case some cities overlapped
combined_geocoded_df.drop_duplicates(subset='city.name', inplace=True)

# Drop entries where geocoding failed
combined_geocoded_df = combined_geocoded_df.dropna(subset=['latitude', 'longitude', 'country'])

print(f"Total unique geocoded cities: {len(combined_geocoded_df)}")

Собираем первый финальный датасет для работы с картой, не забываем указать маркер, что данные от меня 'First Dataset'

In [ ]:
# Assuming city_names_df contains the city names from the first dataset
# Merge geocoded data with the first dataset
merged_df_first = city_names_df.merge(
    combined_geocoded_df[['city.name', 'latitude', 'longitude', 'country']],
    left_on='city.name',
    right_on='city.name',
    how='left'
)

# Add a column to indicate the dataset source
merged_df_first['dataset'] = 'First Dataset'

Получилось что-то такое:

city.name   latitude  longitude country        dataset
0  Saint Petersburg  59.960674  30.158655  Россия  First Dataset
1  Saint Petersburg  59.960674  30.158655  Россия  First Dataset
2  Saint Petersburg  59.960674  30.158655  Россия  First Dataset
3  Saint Petersburg  59.960674  30.158655  Россия  First Dataset
4  Saint Petersburg  59.960674  30.158655  Россия  First Dataset

latitude     longitude
count  39523.000000  39523.000000
mean      55.449206     37.748792
std        7.107904     26.246781
min      -51.778836   -176.174022
25%       54.726141     30.158655
50%       55.625578     35.895242
75%       59.960674     37.729777
max       78.065362    177.506092

In [ ]:
top_10_cities = merged_df_first['city.name'].value_counts().head(10)
print(top_10_cities)

city.name
Saint Petersburg    14428
Moscow               8738
Kyiv                  615
Yekaterinburg         415
Novosibirsk           406
Minsk                 403
Rostov-on-Don         296
Krasnodar             278
Nizhny Novgorod       278
Kazan                 262
Name: count, dtype: int64

То же самое делаем для Васиных данных, его метка 'Second Dataset'

In [ ]:
# Prepare the city names DataFrame for the new dataset
new_city_names_df = new_city_names.to_frame().reset_index(drop=True)
new_city_names_df.rename(columns={'city_name_eng': 'city.name'}, inplace=True)

# Merge geocoded data with the new dataset
merged_df_new = new_city_names_df.merge(
    combined_geocoded_df[['city.name', 'latitude', 'longitude', 'country']],
    left_on='city.name',
    right_on='city.name',
    how='left'
)

# Add a column to indicate the dataset source
merged_df_new['dataset'] = 'Second Dataset'

Получилось что-то вот такое

city.name   latitude  longitude country         dataset
0        Barnaul  53.347549  83.778845  Россия  Second Dataset
1         Alejsk  52.500000  82.783333  Россия  Second Dataset
2          Bijsk  52.516666  85.166664  Россия  Second Dataset
3        Zarinsk  53.707915  84.934861  Россия  Second Dataset
4  Kamen'-na-Obi  53.799999  81.333336  Россия  Second Dataset

In [ ]:
top_10_cities = merged_df_new['city.name'].value_counts().head(10)
print(top_10_cities)

latitude   longitude
count  624.000000  624.000000
mean    54.240179   56.832416
std      5.065075   27.300984
min     42.057858   19.916666
25%     52.049999   38.817731
50%     54.995445   46.141666
75%     56.800016   61.636249
max     69.333336  158.649994

Склеми финальные датасеты для карты (те, что с метками), **удалим города, которые не вышло закодировать**

In [ ]:
# Combine both datasets
merged_df_combined = pd.concat([merged_df_first, merged_df_new], ignore_index=True)

# Drop entries without geocoding information
merged_df_combined = merged_df_combined.dropna(subset=['latitude', 'longitude', 'country'])

### Рисуем карту

In [ ]:
import folium

# Calculate the central point for the map
map_center = [merged_df_combined['latitude'].mean(), merged_df_combined['longitude'].mean()]

# Create a Folium map
city_map = folium.Map(location=map_center, zoom_start=2)

# Define colors for each dataset
dataset_colors = {
    'First Dataset': 'blue',
    'Second Dataset': 'green'
}

# Add city markers to the map
for idx, row in merged_df_combined.iterrows():
    folium.CircleMarker(
        location=(row['latitude'], row['longitude']),
        radius=3,
        popup=f"{row['city.name']}, {row['country']} ({row['dataset']})",
        color=dataset_colors.get(row['dataset'], 'gray'),
        fill=True,
        fill_color=dataset_colors.get(row['dataset'], 'gray')
    ).add_to(city_map)

# Save the map to an HTML file
city_map.save('combined_city_map_2025.html')
print("Combined map has been saved to 'combined_city_map.html'.")